In [6]:
import pandas as pd
import numpy as np
import glob

In [7]:
# 1달치 데이터 가져옴
item_list = glob.glob('../rawdata/2014-01*')
for idx, p in enumerate(item_list):
    df = pd.read_csv(p, encoding = 'cp949')
    if idx == 0:
        all_df = df
    else:
        all_df = pd.concat([all_df, df])

In [8]:
all_df.shape

(2800079, 8)

In [9]:
item_dict =item_info.set_index('pum_id').to_dict()['pum_name']

NameError: name 'item_info' is not defined

In [ ]:
item_info = pd.read_csv('../rawdata/item_info.csv')
all_df['pum_name'] =all_df['pum_id'].map(item_dict)

In [ ]:
# 총 73종류
all_df.pum_name.value_counts()

pum_name
유아복      110676
회화용구      94600
아동화       87748
필기구       75004
스웨터       67605
          ...  
탄산음료       3681
이유식        3619
김          3332
북어채        3306
기초화장품       159
Name: count, Length: 73, dtype: int64

In [ ]:
# 구조 : good_name 을 활용해 pum_name 을 분류

all_df.head()

,collect_day,good_id,pum_id,pum_name,good_name,sales_price,discount_price,benifit
0,2014-01-01,3740.0,E061100,습기제거제,알뜰상품_제습제_4입,4650.0,NaN,NaN
1,2014-01-01,6847.0,L012040,치약,비손_예스미백치약_120G,770.0,NaN,NaN
2,2014-01-01,156664.0,L012080,화장지,해피트리_3겹천연펄프화장지_35M*12R(S),5800.0,NaN,NaN
3,2014-01-01,269074.0,L012180,구강세정제,리스테린 쿨 민트/후레쉬버스트/내추럴 시트러스/티쓰앤껌 750mlx2개 초특가,7800.0,NaN,NaN
4,2014-01-01,356548.0,C021010,아동화,Allie Ballet Flat,57140.0,NaN,NaN


In [10]:
idx2pum = {idx: pum_id for idx, pum_id in enumerate(all_df['pum_id'].unique())}
pum2idx = {v : k for k, v in idx2pum.items()}

In [ ]:
all_df['pid'] = all_df['pum_id'].map(pum2idx)

In [ ]:
rawdata = all_df[['good_name','pid']]

In [ ]:
rawdata.head()

,good_name,pid
0,알뜰상품_제습제_4입,0
1,비손_예스미백치약_120G,1
2,해피트리_3겹천연펄프화장지_35M*12R(S),2
3,리스테린 쿨 민트/후레쉬버스트/내추럴 시트러스/티쓰앤껌 750mlx2개 초특가,3
4,Allie Ballet Flat,4


# 기본 한국어분류모델을 활용해 분류 baseline 을 생성하기

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import torch.optim as optim

In [59]:
!pip install git+https://github.com/huggingface/transformers

In [1]:
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("team-lucid/deberta-v3-base-korean")
model = DebertaV2ForSequenceClassification.from_pretrained("team-lucid/deberta-v3-base-korean",num_label = )

inputs = tokenizer("안녕, 세상!", return_tensors="pt")
outputs = model(**inputs)

tokenizer_config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

C:\Users\jinseulpark\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jinseulpark\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/1.53M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at team-lucid/deberta-v3-base-korean and are newly initialized: ['pooler.dense.bias', 'classifier.weight', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
inputs = tokenizer(["안녕, 세상!","반가워 무슨일이야"], return_tensors="pt")
outputs = model(**inputs)
outputs

In [ ]:
class mydataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        token = tokenizer(self.X[idx], padding = 'max_length', max_length = 96,truncation = True, return_tensors = 'pt') 
        label = torch.tensor(self.y[idx])
        return {'feature': token, 'label': label}

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test =train_test_split(rawdata['good_name'].values, rawdata['pid'].values, test_size = 0.3, random_state = 1205)

In [30]:
X_train_ds = mydataset(X_train, y_train)
X_test_ds = mydataset(X_test, y_test)

In [33]:
X_train[10:11]

array(['[블루테일(아동)] ANH5PT02GY 반통고무줄허리우븐팬츠'], dtype=object)

In [39]:
tokenizer('[블루테일(아동)] ANH5PT02GY 반통고무줄허리우븐팬츠')

TypeError: 'BertTokenizer' object is not callable

In [35]:
[tokenizer(x, padding = 'max_length', max_length = 96,truncation = True, return_tensors = 'pt') for x in X_train[10:11]] 

TypeError: 'BertTokenizer' object is not callable

In [31]:
X_train_ds[0:2]

TypeError: 'BertTokenizer' object is not callable

In [ ]:
DataLoader(X_train_ds)